# Database 와 SQL(Structured Query Language, 구조화 질의어) 사용하기

- ``emailbox-short.txt`` FILE 을 이용하여 sqlite db 생성  


- basic ``CRUD`` (Create, Retrieve, Update, Delete) operation 을 Python 과 SQL 을 이용하여 수행

## 기본 SQL 문법

### 테이블 생성 (CREATE)

- CREATE TABLE 테이블명 (필드명 자료형, ...)

### 데이터조회하기 (SELECT)

- SELECT * FROM 테이블명
- SELECT 열명1, 열명2 FROM 테이블명

### 테이블 구조 참조하기 (DESC)

- DESC 테이블명

### 검색 조건 지정하기 (WHERE)

- SELECT 열1, 열2 FROM 테이블명 WHERE 조건식

### 레코드 추가(INSERT)

- INSERT INTO 테이블명(필드명, ...)  VALUES (값, ...)

### 레코드 삭제(DELETE)

- DELETE FROM 테이블명 WHERE 조건

### 데이터 변경(UPDATE)

- UPDATE 테이블명 SET (필드명=값, ...) WHERE 조건

## db 연결 및 cursor 생성

### connection methods

- connect( ) 를 이용하여 SQLite database 와의 connection object 생성


- cursor( ) - 이 connection 을 이용한 cursor object 를 반환. cursor 는 수행할 SQL 문의 단위가 된다.


- commit( ) - database 에 transaction 반영 


- rollback( ) - transaction 의 starting point 로 되돌아감 


- close( ) - database 와의 연결을 종료

### cursor methods

- execute( ) - string 으로 sql 문장 수행 


- executemany( ) - tuple 의 list 로 여러개 sql 일괄 수행


- fetchone( ) - query result set 에서 다음 row 를 fetch  


- fetchall( ) - query result set 의 나머지 row 를 모두 fetch  

In [1]:
#pip install pymysql

In [1]:
#----vanila mysql connect --------------------------------
import pymysql.cursors

con = pymysql.connect(host='localhost', user='yjoh', password='dreamer01', db='test', charset='utf8mb4')
cur = con.cursor()

### Table 생성

| column | type | 
|:---:|:---:|
| email | TEXT | 
| count  | INTEGER |  

In [3]:
try:        
    cur.execute("DROP TABLE IF EXISTS Counts")
    cur.execute("CREATE TABLE Counts (email TEXT, count INTEGER)")
    print ('table created successfully')
except Exception as e:
    print ('error in operation, ', e)
    con.rollback()
    con.close()

table created successfully


In [4]:
sql = "DESC Counts"
cur.execute(sql)
cur.fetchall()

(('email', 'text', 'YES', '', None, ''), ('count', 'int', 'YES', '', None, ''))

- `From: ` 으로 시작하는 record 에서 email id 를 추출하여 db 에 건수 누적 

- SQL 문에서 `?` 는 placeholder 를 의미. 여러개가 올 수 있으므로 tuple 로 mapping 시킨다

In [5]:
fname = 'emailbox-short.txt'
fh = open(fname)

for line in fh:
    if not line.startswith('From: '):
        continue
    pieces = line.split()
    email = pieces[1]
    cur.execute('SELECT count FROM Counts WHERE email = %s', (email,))
    row = cur.fetchone()
    if row is None:
        cur.execute('INSERT INTO Counts (email, count) VALUES (%s, 1)', 
                        (email,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE email = %s', 
                        (email,))
        
    con.commit()

### 입력된 data 확인

In [6]:
sql = 'SELECT email, count FROM Counts ORDER BY count DESC LIMIT 10'
cur.execute(sql)

10

In [7]:
for row in cur.fetchall():
    print(row[0], row[1])

cwen@iupui.edu 5
zqian@umich.edu 4
david.horwitz@uct.ac.za 4
louis@media.berkeley.edu 3
gsilver@umich.edu 3
stephen.marquard@uct.ac.za 2
rjlowe@iupui.edu 2
wagnermr@iupui.edu 1
antranig@caret.cam.ac.uk 1
gopal.ramasammycook@gmail.com 1


In [8]:
cur.execute('SELECT email, count FROM Counts')
cur.fetchall()

(('stephen.marquard@uct.ac.za', 2),
 ('louis@media.berkeley.edu', 3),
 ('zqian@umich.edu', 4),
 ('rjlowe@iupui.edu', 2),
 ('cwen@iupui.edu', 5),
 ('gsilver@umich.edu', 3),
 ('wagnermr@iupui.edu', 1),
 ('antranig@caret.cam.ac.uk', 1),
 ('gopal.ramasammycook@gmail.com', 1),
 ('david.horwitz@uct.ac.za', 4),
 ('ray@media.berkeley.edu', 1))

In [9]:
cur.execute('SELECT email, count FROM Counts')
cur.fetchone()

('stephen.marquard@uct.ac.za', 2)

In [10]:
cur.fetchone()

('louis@media.berkeley.edu', 3)

In [11]:
cur.fetchone()

('zqian@umich.edu', 4)

In [12]:
cur.fetchall()

(('rjlowe@iupui.edu', 2),
 ('cwen@iupui.edu', 5),
 ('gsilver@umich.edu', 3),
 ('wagnermr@iupui.edu', 1),
 ('antranig@caret.cam.ac.uk', 1),
 ('gopal.ramasammycook@gmail.com', 1),
 ('david.horwitz@uct.ac.za', 4),
 ('ray@media.berkeley.edu', 1))

In [13]:
sql = 'INSERT INTO Counts (email, count) VALUES (%s, %s);'

In [14]:
emails=[('test1@test.com', 100), 
             ('test2@test.com', 200), 
             ('test3@test.com', 300)]

cur.executemany(sql, emails)

3

In [15]:
cur.execute('SELECT email, count FROM Counts')
cur.fetchall()

(('stephen.marquard@uct.ac.za', 2),
 ('louis@media.berkeley.edu', 3),
 ('zqian@umich.edu', 4),
 ('rjlowe@iupui.edu', 2),
 ('cwen@iupui.edu', 5),
 ('gsilver@umich.edu', 3),
 ('wagnermr@iupui.edu', 1),
 ('antranig@caret.cam.ac.uk', 1),
 ('gopal.ramasammycook@gmail.com', 1),
 ('david.horwitz@uct.ac.za', 4),
 ('ray@media.berkeley.edu', 1),
 ('test1@test.com', 100),
 ('test2@test.com', 200),
 ('test3@test.com', 300))

In [16]:
sql = """
CREATE TABLE people
  (
    first_name VARCHAR(20),
    last_name VARCHAR(20),
    age INT
  )
"""

try:        
    cur.execute("DROP TABLE IF EXISTS people")
    cur.execute(sql)
    print ('table created successfully')
except Exception as e:
    print ('error in operation, ', e)
    con.rollback()
    con.close()

table created successfully


In [17]:
sql = """
INSERT INTO people(first_name, last_name, age)
VALUES ('Tina', 'Belcher', 13);
"""
cur.execute(sql)

1

In [18]:
sql = "SELECT * FROM people"
cur.execute(sql)
cur.fetchall()

(('Tina', 'Belcher', 13),)

In [19]:
sql = """
INSERT INTO people(age, last_name, first_name)
VALUES (42, 'Belcher', 'Bob');
"""
cur.execute(sql)

1

In [20]:
sql = """
INSERT INTO people(first_name, last_name, age)
VALUES('Linda', 'Belcher', 45)
              ,('Phillip', 'Frond', 38)
              ,('Calvin', 'Fischoeder', 70);
"""
cur.execute(sql)

3

In [21]:
sql = "SELECT * FROM people"
cur.execute(sql)
cur.fetchall()

(('Tina', 'Belcher', 13),
 ('Bob', 'Belcher', 42),
 ('Linda', 'Belcher', 45),
 ('Phillip', 'Frond', 38),
 ('Calvin', 'Fischoeder', 70))

## CRUD (Create, Read, Update, Delete)

In [22]:
sql = """
CREATE TABLE cats 
  ( 
     cat_id INT NOT NULL AUTO_INCREMENT, 
     name   VARCHAR(100), 
     breed  VARCHAR(100), 
     age    INT, 
     PRIMARY KEY (cat_id) 
  );
"""
try:        
    cur.execute("DROP TABLE IF EXISTS cats")
    cur.execute(sql)
    print ('table created successfully')
except Exception as e:
    print ('error in operation, ', e)
    con.rollback()
    con.close()

table created successfully


In [23]:
sql = "SHOW TABLES"
cur.execute(sql)

7

In [24]:
cur.fetchall()

(('cats',),
 ('cats4',),
 ('counts',),
 ('employees',),
 ('people',),
 ('unique_cats',),
 ('unique_cats2',))

### CREATE

In [25]:
sql = """
INSERT INTO cats(name, breed, age) 
VALUES ('Ringo', 'Tabby', 4),
       ('Cindy', 'Maine Coon', 10),
       ('Dumbledore', 'Maine Coon', 11),
       ('Egg', 'Persian', 4),
       ('Misty', 'Tabby', 13),
       ('George Michael', 'Ragdoll', 9),
       ('Jackson', 'Sphynx', 7);
"""
cur.execute(sql)

7

### READ

In [26]:
sql = "SELECT * FROM cats"

cur.execute(sql)
cur.fetchall()

((1, 'Ringo', 'Tabby', 4),
 (2, 'Cindy', 'Maine Coon', 10),
 (3, 'Dumbledore', 'Maine Coon', 11),
 (4, 'Egg', 'Persian', 4),
 (5, 'Misty', 'Tabby', 13),
 (6, 'George Michael', 'Ragdoll', 9),
 (7, 'Jackson', 'Sphynx', 7))

In [27]:
sql = "SELECT name, age FROM cats"
cur.execute(sql)
cur.fetchall()

(('Ringo', 4),
 ('Cindy', 10),
 ('Dumbledore', 11),
 ('Egg', 4),
 ('Misty', 13),
 ('George Michael', 9),
 ('Jackson', 7))

## where 조건

In [28]:
sql = "SELECT * FROM cats WHERE age=4"
cur.execute(sql)
cur.fetchall()

((1, 'Ringo', 'Tabby', 4), (4, 'Egg', 'Persian', 4))

In [29]:
sql = "SELECT * FROM cats WHERE name='Egg'"
cur.execute(sql)
cur.fetchall()

((4, 'Egg', 'Persian', 4),)

cat_id 와 age 가 같은 조건의 검색

In [30]:
sql = "SELECT * FROM cats WHERE cat_id=age"
cur.execute(sql)
cur.fetchall()

((4, 'Egg', 'Persian', 4), (7, 'Jackson', 'Sphynx', 7))

## Alias

In [31]:
sql = "SELECT cat_id AS id, name FROM cats"
cur.execute(sql)
cur.fetchall()

((1, 'Ringo'),
 (2, 'Cindy'),
 (3, 'Dumbledore'),
 (4, 'Egg'),
 (5, 'Misty'),
 (6, 'George Michael'),
 (7, 'Jackson'))

In [32]:
con.commit()

## Update

In [33]:
sql = "SELECT * FROM cats"
cur.execute(sql)
cur.fetchall()

((1, 'Ringo', 'Tabby', 4),
 (2, 'Cindy', 'Maine Coon', 10),
 (3, 'Dumbledore', 'Maine Coon', 11),
 (4, 'Egg', 'Persian', 4),
 (5, 'Misty', 'Tabby', 13),
 (6, 'George Michael', 'Ragdoll', 9),
 (7, 'Jackson', 'Sphynx', 7))

In [34]:
sql = "UPDATE cats SET breed='Shorthair' WHERE breed='Tabby'"
cur.execute(sql)

2

In [35]:
sql = "SELECT * FROM cats"
cur.execute(sql)
cur.fetchall()

((1, 'Ringo', 'Shorthair', 4),
 (2, 'Cindy', 'Maine Coon', 10),
 (3, 'Dumbledore', 'Maine Coon', 11),
 (4, 'Egg', 'Persian', 4),
 (5, 'Misty', 'Shorthair', 13),
 (6, 'George Michael', 'Ragdoll', 9),
 (7, 'Jackson', 'Sphynx', 7))

In [36]:
con.commit()

## CRUD 연습

### 옷장 속의 shirts 재고 관리

### shirts TABLE 생성

- shirt_id : 셔츠 ID  --> PRIMARY KEY
- article : 셔츠 종류  
- color : 셔츠 색깔  
- shirt_size : 셔츠 사이즈  
- last_worn : 마지막 입은 날부터 경과 일수

In [37]:
sql = """
CREATE TABLE shirts
  (
    shirt_id INT NOT NULL AUTO_INCREMENT,  
    article VARCHAR(100),
    color VARCHAR(100),
    shirt_size VARCHAR(100),
    last_worn INT,
    PRIMARY KEY(shirt_id)
  );
"""

try:        
    cur = con.cursor()
    cur.execute(" DROP TABLE IF EXISTS shirts")
    cur.execute(sql)
    print ('table created successfully')
except Exception as e:
    print ('error in operation, ', e)

table created successfully


- executemany 로 여러개의 record를 batch로 입력

In [41]:
sql = "INSERT INTO shirts (article, color, shirt_size, last_worn) VALUES(%s, %s, %s, %s)"
shirts_records = [
('t-shirt', 'white', 'S', 10),
('t-shirt', 'green', 'S', 200),
('polo shirt', 'black', 'M', 10),
('tank top', 'blue', 'S', 50),
('t-shirt', 'pink', 'S', 0),
('polo shirt', 'red', 'M', 5),
('tank top', 'white', 'S', 200),
('tank top', 'blue', 'M', 15)
]

cur.executemany(sql, shirts_records)

8

- 입력된 record 확인

In [42]:
sql = "SELECT * FROM shirts"
cur.execute(sql)
cur.fetchall()

((1, 't-shirt', 'white', 'S', 10),
 (2, 't-shirt', 'green', 'S', 200),
 (3, 'polo shirt', 'black', 'M', 10),
 (4, 'tank top', 'blue', 'S', 50),
 (5, 't-shirt', 'pink', 'S', 0),
 (6, 'polo shirt', 'red', 'M', 5),
 (7, 'tank top', 'white', 'S', 200),
 (8, 'tank top', 'blue', 'M', 15))

- 새로운 record 추가

In [44]:
sql = """
INSERT INTO shirts (color, article, shirt_size, last_worn) 
        VALUES('purple', 'polo shirt', 'medium', 50)
"""
cur.execute(sql)

1

In [45]:
sql = "SELECT * FROM shirts"
cur.execute(sql)
cur.fetchall()

((1, 't-shirt', 'white', 'S', 10),
 (2, 't-shirt', 'green', 'S', 200),
 (3, 'polo shirt', 'black', 'M', 10),
 (4, 'tank top', 'blue', 'S', 50),
 (5, 't-shirt', 'pink', 'S', 0),
 (6, 'polo shirt', 'red', 'M', 5),
 (7, 'tank top', 'white', 'S', 200),
 (8, 'tank top', 'blue', 'M', 15),
 (9, 'polo shirt', 'purple', 'medium', 50))

- medium size 의 shirts 만 검색

In [47]:
sql = "SELECT article, color, shirt_size, last_worn FROM shirts WHERE shirt_size='M'"
cur.execute(sql)
cur.fetchall()

(('polo shirt', 'black', 'M', 10),
 ('polo shirt', 'red', 'M', 5),
 ('tank top', 'blue', 'M', 15))

- polo shirts의 size를 L로 변경

In [48]:
sql = "UPDATE shirts SET shirt_size='L' WHERE article='polo shirt'"
cur.execute(sql)

3

In [49]:
sql = "SELECT article, color, shirt_size, last_worn FROM shirts WHERE article='polo shirt'"
cur.execute(sql)
cur.fetchall()

(('polo shirt', 'black', 'L', 10),
 ('polo shirt', 'red', 'L', 5),
 ('polo shirt', 'purple', 'L', 50))

흰색 shirts 의 color를 'off white' 로, shirts_size를 XS로 변경

In [50]:
sql = "UPDATE shirts SET color='off white', shirt_size='XS' WHERE color='white'"
cur.execute(sql)

2

In [53]:
sql = "SELECT article, color, shirt_size, last_worn FROM shirts WHERE color='off white'"
cur.execute(sql)
cur.fetchall()

(('t-shirt', 'off white', 'XS', 10), ('tank top', 'off white', 'XS', 200))

'tank top' shirts를 삭제

In [54]:
sql = "DELETE FROM shirts WHERE article='tank top'"
cur.execute(sql)

3

In [55]:
sql = "SELECT * FROM shirts"
cur.execute(sql)
cur.fetchall()

((1, 't-shirt', 'off white', 'XS', 10),
 (2, 't-shirt', 'green', 'S', 200),
 (3, 'polo shirt', 'black', 'L', 10),
 (5, 't-shirt', 'pink', 'S', 0),
 (6, 'polo shirt', 'red', 'L', 5),
 (9, 'polo shirt', 'purple', 'L', 50))

- TABLE의 전체 shirts 삭제

In [57]:
sql = "DELETE FROM shirts"
cur.execute(sql)

sql = "SELECT * FROM shirts"
cur.execute(sql)
cur.fetchall()

()

- TABLE 삭제

In [59]:
sql = "SHOW TABLES"
cur.execute(sql)
cur.fetchall()

(('cats',),
 ('cats4',),
 ('counts',),
 ('employees',),
 ('people',),
 ('shirts',),
 ('unique_cats',),
 ('unique_cats2',))

In [60]:
sql = "DROP TABLE shirts"
cur.execute(sql)

sql = "SHOW TABLES"
cur.execute(sql)
cur.fetchall()

(('cats',),
 ('cats4',),
 ('counts',),
 ('employees',),
 ('people',),
 ('unique_cats',),
 ('unique_cats2',))

In [61]:
con.commit()
con.close()

## 연습문제

- 다음 table 을 database 로 생성  

- executemany( ) 를 이용하여 A1~A3 까지의 data 를 일괄 insert

- 전체를 조회  

- A4 를 추가로 insert  

- A3 의 나이를 59 로 변경  

- A1 을 삭제  

- 전체를 조회


|학생번호| 성명 |나이 |성적|
|:------:|:-----:|---|----|
|A1|홍길동|20|70|
|A2|유병길|16|80|
|A3|김길수|29|90|
|A4|박재만|25|86|